The Transformer is the latest advance in Deep Learning architectures that has driven most state-of-the-art progress in NLP since it was first presented in ['Attention is All You Need'](https://arxiv.org/abs/1706.03762). Since then, ever larger models are being made, with parameters running into the billions. 

> Side-note: I think we're inflection point in ML with OpenAI's release of their API - everyone now has easy access to these state-of-the-art language models, we're gonna see an explosion of use-cases + value creation


There's a lot of greats resources with visualisations to help understand the architecture which I'll come back to. First, a brief introduction to what makes Transformers so powerful:

*   *Self-attention*: a mechanism allowing us to learn contextual relationships between different elements in our input sequence, replacing the need for sequential structure (from RNN/LSTM cells).
*   *Multi-headed attention*: multiple heads of the model carry out self-attnetion, attending to information jointly at different parts of the sequence from different subspaces. This allows us to learn a variety of features of language + means the model can scale efficiently with large datasets + unsupervised learning.
* *Transfer learning*: Transformers use the knowledge extracted from a prior setting (usually in the form a language model), which can be unsupervised, then apply or *transfer* to a specific domain, where labelled data is available. This allows a large rich corpus of text to be used in the first pre-training stage, before the model is fine-tuned on custom data. 

*insert pre training photo*

In this post, we'll look at how to fine tune a pre-trained model for the task fo sentiment analysis using Hugging Face's [Transformer](https://huggingface.co/transformers/pretrained_models.html) library, that gives simple access to many of the top transformed-based models (*BERT*, *GPT-2*, *XLNet* etc).  We'll use *DistilBert* here, a lightweight version of the famous *BERT* model with 66 million parameters that's slightly easier to run on a single Colab GPU.

BERT stands for Bidirectional Encoder Representations from Transformers. It uses a *masked* language model where 15% of a sequence's tokens are randomly masked, then the model learns to predict, given a token, what came before *or* after it (the bi-dircectional part). In addition, it has a next sentence prediction objective (did this sentence come after a previous one). BERT differs from a more standard *casual* language model, that predicts the most likely next token in the sequence in a left-to-right direction.


## Setup

In [1]:
!pip install transformers

In [3]:
#from google.colab import drive # import drive from google colab

In [4]:
#ROOT = "/content/drive"     # default location for the drive
#print(ROOT)                 # print content of ROOT (Optional)

#drive.mount(ROOT)           # we mount the google drive at /content/drive

In [2]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [4]:
import transformers
import torch
from transformers import DistilBertModel, DistilBertTokenizer, AdamW, get_linear_schedule_with_warmup
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM


from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

from os import path
import requests
import gzip
import zipfile
import numpy as np
from collections import defaultdict

# RANDOM_SEED = 0
# np.random.seed(RANDOM_SEED)
# torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-multilingual-cased")
model = AutoModelForMaskedLM.from_pretrained("distilbert/distilbert-base-multilingual-cased")


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

## Loading our Data

For the task of sentiment analysis our model takes a sentence as input and outputs one of five classes representing sentiments (very negative, negative, neutral, positive, very positive). The Stanford Sentiment Treebank (SST-5) is the best-known dataset for this, composed of 11855 such sentences with labels 1-5 already split into train, validation and test sets (of sizes 8544, 1101 and 2210). 

Let's download the dataset, then split into train/val/test sets.

In [7]:
import pandas as pd

# Load the data from the CSV file
data = pd.read_csv("/kaggle/input/mtgcn-data/modified_file.csv")


In [9]:
# Separate features (X) and labels (y)
X = data["review"].tolist()
y = data["sentiment"].tolist()


In [10]:


# Split the data into train, validation, and test sets
from sklearn.model_selection import train_test_split

# Split into train and temporary set (to be further split into validation and test)
X_train_temp, X_test, y_train_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Further split the temporary set into validation and train
X_train, X_val, y_train, y_val = train_test_split(X_train_temp, y_train_temp, test_size=0.25, random_state=42)

# Now you have X_train, y_train, X_val, y_val, X_test, y_test ready for training and evaluation


We need to turn each sequence of words into tokens that serve as inputs into our model. The `DistilBertTokenizer` object does just that. We can see what the tokenizer does to the first sentence in our training set.


In [11]:
PRE_TRAINED_MODEL_NAME = 'distilbert/distilbert-base-multilingual-cased'

#tokenizer = DistilBertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [12]:
sample_txt = str(X_train[0])
tokens = tokenizer.tokenize(sample_txt)
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(f' Sentence: {sample_txt}')
print(f'   Tokens: {tokens}')
print(f'Token IDs: {token_ids}')

 Sentence: ఎన్నికల్లో ప్రజలు తెలుగుదేశం పార్టీ కి గట్టి బుద్ది చెప్పబోతున్నారని ఆయన అన్నారు
   Tokens: ['ఎ', '##న్ని', '##క', '##ల్లో', 'ప్రజలు', 'తెలుగు', '##దేశం', 'ప', '##ార', '##్', '##టీ', 'కి', 'గ', '##ట్టి', 'బ', '##ు', '##ద్', '##ది', 'చ', '##ె', '##ప్', '##ప', '##బ', '##ో', '##తున్న', '##ార', '##ని', 'ఆయన', 'అ', '##న్న', '##ారు']
Token IDs: [1195, 31114, 15280, 46890, 108356, 45419, 105147, 1220, 67787, 24835, 37551, 11022, 1203, 60660, 1222, 15697, 76859, 19541, 1205, 37637, 50445, 48317, 111337, 21120, 44275, 67787, 13907, 39567, 1188, 37218, 27374]


The model needs to account for a few special tokens, namely the start + end of a sentence, unknown words and lastly for padding (each sentence has a different length, not well suited to feed into batches for a deep learning model so we set a suitable max length, then pad shorter sentences up to that length with a padding token.)  All this word is done for us using the `encode_plus` method, which we use to build our `Dataset` object.

In [13]:
class SST_Dataset(Dataset):
    def __init__(self, ys, Xs, tokenizer, max_len):
        self.targets = ys
        self.reviews = Xs
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, idx):
        review = str(self.reviews[idx])
        target = self.targets[idx]
        encoding = self.tokenizer.encode_plus(
          review,
          add_special_tokens=True,
          max_length=self.max_len,
          return_token_type_ids=False,
          pad_to_max_length=True,
          return_attention_mask=True,
          return_tensors='pt',
          truncation=True
        )
        return {
          'review_text': review,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten(),
          'targets': torch.tensor(target, dtype=torch.long)
        }

Next we create our `Dataloader` objects for training, validation and testing. For each item in the dataset we need the encoded input tokens, masks for where the sentence is not padded and the target value.

In [14]:
def create_data_loader(ys, Xs, tokenizer, max_len, batch_size):
    ds = SST_Dataset(ys, Xs, tokenizer, max_len)
    return DataLoader(ds, batch_size=batch_size)

BATCH_SIZE = 16
MAX_LEN = 128

train_data_loader = create_data_loader(y_train, X_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(y_val, X_val, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(y_test, X_test, tokenizer, MAX_LEN, BATCH_SIZE)

## Constructing our model

Now we'ready to build our simple sentiment classification model: we use the output of the `DistilBertModel` - of size 768 - as input into a single fully-connected layer. Dropout is important here for a model with so many parameters (discussed below). (Hugging Face also provide some inbuilt models for downstream tasks that we could have used such as `BertForSequenceClassification` or `BertForQuestionAnswering`)


In [15]:
class SentimentClassifier(nn.Module):
  def __init__(self, n_classes=3):
    super(SentimentClassifier, self).__init__()
    self.bert = DistilBertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
    self.drop = nn.Dropout(p=0.3)
    self.fc = nn.Linear(self.bert.config.hidden_size, n_classes)

  def forward(self, input_ids, attention_mask):
    output = self.bert(input_ids, attention_mask)
    output= output[0][:,0]
    output = self.drop(output)
    return self.fc(output)

The BERT authors had some recommendations for hyperparameters when it comes to fine-tuning:

*   *Batch size*: 16, 32
*   *Learning rate (Adam)*: 5e-5, 3e-5, 2e-5
*   *Number of epochs*: 2, 3, 4

We'll largely stick with these - note that the number of epochs is a lot lower than you might expect for a Deep Learning model. This is since we can easily overfit to the training set with many parameters. We'll check for this by calculating both the training and validation accuracy at each epoch. You can find out more about the Hugging Face's optimisers [here](https://huggingface.co/transformers/main_classes/optimizer_schedules.html).

In [16]:
# initialise model
model = SentimentClassifier()

EPOCHS = 5
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_data_loader)*EPOCHS
scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=50,
  num_training_steps=total_steps
)
loss_fn = nn.CrossEntropyLoss().to(device)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Let’s continue with writing our helper functions for training our model. 

In [17]:
def evalModel(model, data_loader, loss_fn, N):
    """Evaluate loss and accuracy of model on data_loader"""
    # set model to evaluation mode
    model = model.eval()
    total_loss = 0
    correct = 0

    with torch.no_grad():
        for d in data_loader:
            # get inputs and target 
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)

            # pass through model + make prediction
            outputs = model(input_ids, attention_mask)
            _, pred = torch.max(outputs, dim=1)

            # update counters
            loss = loss_fn(outputs, targets)
            correct += (pred == targets).sum().item()
            total_loss += loss.item()*len(targets)

    # normalise
    return 100*correct/N, total_loss/N

In [18]:
def trainModel(model, trainDataLoader, valDataLoader, loss_fn, optimizer, scheduler, verbose=True):
    """Train sentiment classifier"""
    # structure to store progress of the model at each epoch
    history = defaultdict(list)
    
    # move the model to the gpu
    model = model.to(device)

    for ep in range(EPOCHS):
        total_loss = 0
        correct = 0
        # set model to train mode so dropout and batch normalisation layers work as expected
        model.train()

        for d in trainDataLoader:
            # get inputs for batch
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)

            # calculate output + loss
            model.zero_grad()
            outputs = model(input_ids, attention_mask)
            loss = loss_fn(outputs.squeeze(), targets.long())

            # take gradient step
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            scheduler.step()

            # update losses
            _, pred = torch.max(outputs, dim=1)
            correct += (pred == targets).sum().item()
            total_loss += loss.item()*len(targets)

        #after each epoch, collect statistics
        history['train_acc'].append(100*correct/len(X_train))
        history['train_loss'].append(total_loss/len(X_train))

        # statistics about the validation set
        val_acc, val_loss = evalModel(model, valDataLoader, loss_fn, len(X_val))
        history['val_acc'].append(val_acc)
        history['vall_loss'].append(val_loss)

        #if validation improved, save new best model
        if history['val_acc'][-1] == max(history['val_acc']):
            print ("=> Saving a new best at epoch:", ep)
            torch.save(model.state_dict(), 'best_model_state.bin')
        
        if verbose:
            print('Epoch {}/{}'.format(ep+1, EPOCHS))
            print('-' * 10)
            print('Train loss {} accuracy {}'.format(history['train_loss'][-1], history['train_acc'][-1]))
            print('Val loss {} accuracy {}'.format(val_loss, val_acc))

    #clean up
    model = model.to(torch.device("cpu"))
    del input_ids, attention_mask, targets, outputs, _, pred

    return model, history

Let's train our model and see how it does on our test set!

In [19]:
%%time
best_model, histories = trainModel(model, train_data_loader, val_data_loader, loss_fn, optimizer, scheduler, verbose=True)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


=> Saving a new best at epoch: 0
Epoch 1/5
----------
Train loss 0.9162647313217672 accuracy 57.602921646746346
Val loss 0.8019359268959326 accuracy 63.721724285104564
=> Saving a new best at epoch: 1
Epoch 2/5
----------
Train loss 0.7397323297020426 accuracy 67.02712957693038
Val loss 0.8055698023757888 accuracy 65.74192630530659
=> Saving a new best at epoch: 2
Epoch 3/5
----------
Train loss 0.62924395664732 accuracy 72.96053879719219
Val loss 0.8558117336502012 accuracy 66.04068857589985
Epoch 4/5
----------
Train loss 0.5440891673659446 accuracy 77.06791880098653
Val loss 0.8869479595809255 accuracy 65.88419405320813
=> Saving a new best at epoch: 4
Epoch 5/5
----------
Train loss 0.489029304089254 accuracy 79.78561942705369
Val loss 0.9197145097454813 accuracy 66.04068857589985
CPU times: user 13min 53s, sys: 15.7 s, total: 14min 9s
Wall time: 14min 10s


In [20]:
test_acc, test_loss = evalModel(best_model.to(device), test_data_loader, loss_fn, len(y_test))

In [21]:
print(test_acc, test_loss)

66.53862569355528 0.914690301290768


In [22]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Evaluate the model on the test set
def evaluate_model(model, test_data_loader, loss_fn):
    model.eval()
    total_loss = 0
    all_predictions = []
    all_targets = []

    with torch.no_grad():
        for batch in test_data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            targets = batch['targets'].to(device)

            outputs = model(input_ids, attention_mask)
            loss = loss_fn(outputs, targets)

            total_loss += loss.item() * len(targets)
            _, predictions = torch.max(outputs, dim=1)

            all_predictions.extend(predictions.cpu().tolist())
            all_targets.extend(targets.cpu().tolist())

    avg_loss = total_loss / len(test_data_loader.dataset)
    return all_predictions, all_targets, avg_loss

# Evaluate the model
test_predictions, test_targets, test_loss = evaluate_model(best_model, test_data_loader, loss_fn)

# Convert to numpy arrays
test_predictions = np.array(test_predictions)
test_targets = np.array(test_targets)

# Calculate precision, recall, and F1 score
precision = precision_score(test_targets, test_predictions, average='weighted')
recall = recall_score(test_targets, test_predictions, average='weighted')
f1 = f1_score(test_targets, test_predictions, average='weighted')

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Precision: 0.6603619444995041
Recall: 0.6653862569355528
F1 Score: 0.6608919320480887


There we have it! We've fine-tuned DistilBert for the task of sentiment classification to over 40% test accuracy in only 5 epochs. We can see that the pre-training step of this Tranformer model produces versatile, useful and high-quality features representing different semantics of language.

However we note that this doesn't get us close to [state-of-the-art](https://paperswithcode.com/sota/sentiment-analysis-on-sst-5-fine-grained) on this dataset (55%) - the important lesson here is that we haven't tuned any hyperparameters so finding the best optimizer, learning-rate, droupout amount, adding hidden-layers + number of epochs is what will improve our model. We use the validation set to see what hyperparameters get the best accuracy on that - this estimates how our model will generalise to the unseen test set (see your favourite Learning Theory textbooka as to why this works).

Remember that during training we're trying to find the optima a (> 66,000,000 dimension) hypersurface - there's going to many minima so finding the best one requires some searching. Hyperparameter tuning is an important part of solving any problem with Machine Learning, one you just can't avoid.

As a final bit of fun, let's see what our model predicts on some raw text - we need to tokenise our custom input then pass it through our trained classifier. Though not a 5 we see the model can correctly identify the review as positive!

In [23]:
review_text = "నాకు చిత్రం బాగా నచ్చింది ."

In [24]:
encoded_review = tokenizer.encode_plus(
  review_text,
  max_length=MAX_LEN,
  add_special_tokens=True,
  return_token_type_ids=False,
  pad_to_max_length=True,
  return_attention_mask=True,
  return_tensors='pt',
  truncation=True
)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [25]:
input_ids = encoded_review['input_ids'].to(device)
attention_mask = encoded_review['attention_mask'].to(device)
output = model(input_ids, attention_mask)
_, prediction = torch.max(output, dim=1)
print(f'Review text: {review_text}')
print(f'Sentiment  : {int(prediction.cpu().detach().numpy())}')

Review text: నాకు చిత్రం బాగా నచ్చింది .
Sentiment  : 2


/tmp/ipykernel_34/2852791618.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(f'Sentiment  : {int(prediction.cpu().detach().numpy())}')
